<a href="https://colab.research.google.com/github/kahinasassi/initial-text-preprocessing--arabic-dialect-/blob/master/pretraitementInitiale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auteur: SASSI KAHINA "ISII"




## *le fichier regroupe les traitements appliqué dans la phase de prétraitement initaile*

In [ ]:

import numpy as np
import pandas as pd
import nltk
from nltk import re
from nltk import defaultdict
import io
from nltk import  word_tokenize
from nltk import  sent_tokenize
from nltk.corpus import stopwords
import codecs
import csv
import pickle
from sklearn.naive_bayes import GaussianNB,MultinomialNB,ComplementNB
from sklearn import metrics
from gensim.parsing.preprocessing import strip_short
import nltk
import phonetics
from nltk.probability import ConditionalFreqDist

from itertools import groupby
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#vérifie si l'encodage est utf_8
def check_utf8(filename): 
        try:
            f = codecs.open(filename, encoding='utf-8', errors='strict')
            for line in f:
                pass
            return 1 #valid utf_8
        except UnicodeDecodeError:
            return 0 #invalid utf_8

# ouvrir un fichier text
def open_txt_file(chemin):
    #open text file atfter checking if its utf_8  encoding    
    check=check_utf8(chemin)
    if(check == 1):
        f = io.open(chemin,'r',encoding='utf8')
    elif(check == 0):
        f = io.open(chemin,'r')
    return f.read()

# convertir un text vers une liste
def text_to_list(chemin):
    text=open_txt_file(chemin)
    liste=[]
    liste=text.split("\n")
    return liste

# ouvrir un fichier pickle
def open_pickle(path):
  with open(path, 'rb') as handle: 
    doc= pickle.load(handle)
    return doc

def save_pickle(variable,filename):
    with open(filename, 'wb') as handle:
         pickle.dump(variable, handle, protocol=pickle.HIGHEST_PROTOCOL)

def is_arabizi(word):
  if len(re.findall(r'^[a-zA-Z0-9]+$',word)) > 0:
    return 1
  else: 
    return 0





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#Suppression des urls , hachtags et tags

In [ ]:
# TRAITEMENT 1


import re
import string


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


  
  
def remove_tags(text):
  return re.sub(r"@(\w+|_|\.)*", ' ', text, flags=re.MULTILINE)
  

def remove_hashtag(text):
  return re.sub(r"#(\w+|_)*", ' ', text, flags=re.MULTILINE)
 

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_numeric


# suppression des lettre répérté

In [ ]:
#charger les deux doctionnaires, francais et anglais pour faire la détection de la langue avant la traduction
dictionnaire_francais=open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/lexique/lexique_fr.pickle")
dictionnaire_anglais=open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/lexique/lexique_eng.pickle")

In [ ]:
def detect_fr(word): #reçoit le mot, retourne vrai s'il est francais, faux sinon
    word_min=word.lower()
    if word_min in dictionnaire_francais:
        return True
    else:
        return False

def detect_en(word): #reçoit le mot, retourne vrai s'il est anglais, faux sinon
    word_min=word.lower()
    if word_min in dictionnaire_anglais:
        return True
    else:
        return False


In [ ]:
#Convertion en miniscule + Suppression les lettres répérés
def remove_repeating_char(text):
  lower_word=text.lower()
  liste=tokenization(lower_word)
  sentence=''
  for word in liste:
    if(detect_en(word) == False):
      if(sentence == ''):
        new=''.join(c for c, _ in groupby(word))# supprime les caractères répété dans un mots(group by retourne une seul occurance de chaque lettre dans le mot)
        sentence=sentence+''+new
      else:
        new=''.join(c for c, _ in groupby(word))
        sentence=sentence+' '+new
    else: # si le mot est anglais ie detecté dans le dictionnaire( good) le mot ne sera traité comme mot originale
      if(sentence == ''):
        sentence=sentence+word
      else:
        sentence=sentence+' '+word
  return sentence

#Suppression de la ponctuation

In [ ]:
def remove_ponctuation(text):
   puncts = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│','!','?', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗',' ️�','▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
   
   x = str(text)
   for punct in puncts:
      if punct in x:
          x = x.replace(punct, f' ')
   return x
def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

#Suppression des imojis

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)
print(remove_emoji("woman with dignity nombre human heart �☝✊"))
#----------------------------------------------------------------------------------------------------------


#load la liste des emojis a partir de facebook
liste_emoji=text_to_list("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/all_emojis.txt")

# la suppression

def remove_imogi(text):  
   x = str(text)
   for punct in liste_emoji:
      if punct in x:
          x = x.replace(punct, f'')
   return x

woman with dignity nombre human heart 


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
def naive_bayes(data):
  # Set up training and test sets by choosing random samples from classes
  X_train, X_test, y_train, y_test = train_test_split(data['text'].values, data['label'].values, test_size=0.20, random_state=0)
  model = MultinomialNB()
  vect = CountVectorizer()
  vect.fit_transform(data['text'].values)  #train the vectorizer, build the vocablury
  tfidf_transformer=TfidfTransformer()
  X_train_tfidf=vect.transform(X_train)
  X_test_vect=vect.transform(X_test)
  model.fit(X_train_tfidf,y_train)
  model.score(X_train_tfidf,y_train)
  y_pred = model.predict(X_test_vect)
  y_pred
  from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
  print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
  print("\nCOnfusion Matrix:\n", confusion_matrix(y_test, y_pred))

#suppression des chiffres isolé

In [ ]:
def remove_numbers(text):
  return  re.sub(r"\b[0-9]+\b", " ",text)

# suppression des mots court

In [ ]:
def remove_short(text):
  lower_word=text.lower()
  liste=tokenization(lower_word)
  sentence=''
  for word in liste:
    if(len(word) == 1):
        new=''
        sentence=sentence+''+new
    else:
      if(sentence == ''):
        new=word
        sentence=sentence+''+new
      else:
        new=word
        sentence=sentence+' '+new
      
  return sentence

tokenization

In [ ]:
def tokenization(text):
    return word_tokenize(text)

#*Execution* et evaluation

In [ ]:
data=open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/donnefusioner.pickle")
#Suppression des instances doublons
data=data.drop_duplicates()
print(len(data))

#supression nan
data.isnull().sum()
data=data.dropna(how='any')
data.isnull().sum()

21054


text     0
label    0
dtype: int64

In [ ]:
data.info()
print('**********************************************************************')
print(data.shape)
print('**********************************************************************')
#print(data.label.value_counts())
print("nbr de val positives = ",len(data.loc[data['label'] == 1].iloc[:,1]))
print("nbr de val negatives = ",len(data.loc[data['label'] == -1].iloc[:,1]))
print("nbr de val neutres = ",len(data.loc[data['label'] == 0].iloc[:,1]))
print("**********************************************************************")
print("les valeurs nuls:\n", data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21023 entries, 0 to 22021
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    21023 non-null  object 
 1   label   21023 non-null  float64
dtypes: float64(1), object(1)
memory usage: 492.7+ KB
**********************************************************************
(21023, 2)
**********************************************************************
nbr de val positives =  8504
nbr de val negatives =  4632
nbr de val neutres =  7877
**********************************************************************
les valeurs nuls:
 text     0
label    0
dtype: int64


# Avant le prétraitement

In [ ]:
for indice  in list(data.index.values): 
    text=str(data['text'][indice])
    data['text'][indice]=text
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 66.56%

COnfusion Matrix:
 [[ 538  238  160    0]
 [ 191  982  427    0]
 [ 225  164 1279    0]
 [   0    1    0    0]]


#suppression des urls

In [ ]:
for indice  in list(data.index.values): 
    text=str(data['text'][indice])
    text=remove_URL(text) 
    data['text'][indice]=text
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Accuracy: 66.54%

COnfusion Matrix:
 [[ 537  239  160    0]
 [ 191  981  428    0]
 [ 225  163 1280    0]
 [   0    1    0    0]]


#suppression des tags

In [ ]:
for indice  in list(data.index.values): 
    text=str(data['text'][indice])
    text=remove_tags(text) 
    data['text'][indice]=text
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Accuracy: 66.54%

COnfusion Matrix:
 [[ 537  239  160    0]
 [ 191  981  428    0]
 [ 225  163 1280    0]
 [   0    1    0    0]]


#suppression des hachtags

In [ ]:
for indice  in list(data.index.values): 
    text=str(data['text'][indice])
    text=remove_hashtag(text) 
    data['text'][indice]=text
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Accuracy: 66.59%

COnfusion Matrix:
 [[ 538  239  159    0]
 [ 189  982  429    0]
 [ 226  162 1280    0]
 [   0    1    0    0]]


#supression des caractères répétés

In [ ]:
for indice in data.index: 
    text=str(data['text'][indice])
    data['text'][indice]=remove_repeating_char(text)
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 67.02%

COnfusion Matrix:
 [[ 537  238  161    0]
 [ 193  985  422    0]
 [ 205  167 1296    0]
 [   0    1    0    0]]


#suppression de la ponctuation

In [ ]:
for indice in data.index: 
    text=str(data['text'][indice])
    data['text'][indice]=remove_punct(text) 
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 67.09%

COnfusion Matrix:
 [[ 534  239  163    0]
 [ 190  987  423    0]
 [ 200  168 1300    0]
 [   0    1    0    0]]


#suppression des mots court

In [ ]:
for indice in data.index: 
    text=str(data['text'][indice])
    data['text'][indice]=remove_short(text)
naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy: 67.09%

COnfusion Matrix:
 [[ 534  239  163    0]
 [ 190  987  423    0]
 [ 200  168 1300    0]
 [   0    1    0    0]]


#suppression des chiffres

In [ ]:
for indice in data.index: 
    text=str(data['text'][indice])
    data['text'][indice]=remove_numbers(text)


# suppression des imogis

In [ ]:
data["text"] = data.text.map(lambda x: remove_emoji(x))

naive_bayes(data)


Accuracy: 67.04%

COnfusion Matrix:
 [[ 538  225  173    0]
 [ 194  980  426    0]
 [ 205  162 1301    0]
 [   0    1    0    0]]


#Traitement des abréviation

## 2 remplacement

Le but: identifier le mot abrégé dans un commentaire et le remplacer par sa signification
- on verifie chaque mot si est une abréviation
- si oui on cherche sa signification dans le fichier
- on remplace dans la phrase

In [ ]:
#load la liste des abréviations 
url="/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/abréviation.csv"
abreviation=pd.read_csv(url)

def replace_abrv(texte):
 word= tokenization(texte) 
 x=str(texte)
 ma_lis=[]
 for val in word:
   if(len(abreviationne(val))!=0):
     for indice  in list(abreviation.index.values): 
       text=str(abreviation['text'][indice])
       if(val== text):
         tt=str(abreviation['abreviation'][indice])
         ma_lis.append(tt)
         x = x.replace(val, tt)  
 return x  


#print(replace_abrv("hi evry bnjr bn mone"))


In [ ]:
for indice  in list(data.index.values): 
    text=str(data['text'][indice])   
    text=replace_abrv(text) 
    data['text'][indice]=text

naive_bayes(data)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Accuracy: 67.09%

COnfusion Matrix:
 [[ 536  237  163    0]
 [ 190  985  425    0]
 [ 201  167 1300    0]
 [   0    1    0    0]]


In [ ]:
save_pickle(data,"/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/dataset finale/Donné trouvé/data_pré_init.pickle")
